In [1]:
import pandas as pd
# import gzip
import numpy as np
# import json
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Video_Games_Resampled.csv')

In [5]:
# df = df.reindex(np.random.permutation(df.index))
# ratingonly = df.shape
# df = df.dropna(subset=['overall'],inplace=True)
# df = df.dropna(subset=['reviewText'],inplace=True)
# df.head()

,Unnamed: 0.1,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,Positive Rating
0,346407,129412,5.0,False,"05 3, 2010",A3DID3K8W8QD9W,B0012LHO46,Sam Hartford,bought mine ebay one amazon work fine generics...,PS2 64mb Memory Card is A-OK,1272844800,NaN,NaN,NaN,1
1,19478,204432,4.0,True,"05 5, 2014",A1SPRD853KYUP1,B002I0K956,Oswaldo Romero,nice,Four Stars,1399248000,NaN,NaN,NaN,1
2,349043,367307,5.0,True,"01 4, 2017",A4SV916591F5Z,B00KY1I0IO,Amazon Customer,great fun addicting,Five Stars,1483488000,NaN,{'Format:': ' Video Game'},NaN,1
3,238716,381621,4.0,True,"02 12, 2016",AWXOF97NP6ZZG,B00O9GPD26,Amazon Customer,surprisingly higher quality figure american ve...,The figure is better than the american version.,1455235200,NaN,NaN,NaN,1
4,416659,456490,5.0,False,"08 5, 2005",A3LOOGDJ7S01LA,B00008J2V0,whiterabbit,highly recommended whole family easy instructi...,"Lots of Fun, Lots to Explore!",1123200000,7,NaN,NaN,1


In [4]:
# ax=sns.countplot(x='overall', data=df)
# ax.bar_label(ax.containers[0])
# plt.grid(visible=True,axis='y',alpha=0.5)
# plt.show()

In [ ]:
# rating_threshold = 4
# df['Positive Rating'] = df['overall'].apply(lambda x: 1 if x>=rating_threshold else 0)
# df['Positive Rating'].head(10)

In [ ]:
# from nltk.corpus import stopwords
# import re
# from nltk.tokenize import word_tokenize, RegexpTokenizer

In [ ]:
# def clean_text(text):
#     # lower text
#     text = text.lower()
#     # remove hyperlinks
#     text = re.compile(r'^https?://', re.IGNORECASE).sub(r'', text)
#     # remove non-letters
#     text = re.sub("[^a-zA-Z]", " ", text)
#     text = word_tokenize(text)
#     # remove stopwords
#     stops = set(stopwords.words("english"))
#     text = [w for w in text if not w in stops]
#     text = " ".join(text)
#     # remove punctuation
#     text = RegexpTokenizer(r'\w+').tokenize(text)
#     text = " ".join(text)
#     # remove words less than 3 letters
#     text = re.sub(r'\b\w{1,2}\b', '', text)
#     return text  



In [ ]:
# df.dropna(subset=['reviewText'],inplace=True)
# df['reviewText'] = df['reviewText'].apply(clean_text)

In [ ]:
# from sklearn.utils import resample

# majority = df[df['Positive Rating']==1]
# minority = df[df['Positive Rating']==0]
# minority_upsampled = resample(minority, replace=True, n_samples=int(len(minority)*1.6), random_state=87)
# majority_downsampled =  resample(majority, replace=True, n_samples=int(len(majority)/2.2), random_state=100)
# resampled_df = pd.concat([majority_downsampled, minority_upsampled])

# print(resampled_df.shape)
# resampled_df['overall'].value_counts()
# resampled_df.to_csv('Video_Games_Resampled.csv')

In [ ]:
# df=pd.read_csv('Video_Games_Resampled.csv')
# df.shape

In [3]:
from sklearn.model_selection import train_test_split

trainx, testx, trainy, testy = train_test_split(df['reviewText'], df['Positive Rating'], test_size=0.23, random_state=42)
print(len(trainx))
print(len(testx))
train = pd.concat([trainx, trainy], axis =1)
test = pd.concat([testx, testy], axis=1)
# train.to_csv('train_data.csv', index=True)
# test.to_csv('test_data.csv',index=True)

263969
78849


In [4]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
# from sklearn.feature_extraction.text import TfidfVectorizer
# import torch.nn.functional as F
import time
from tqdm import tqdm

In [5]:
import spacy
# from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T
# spacy.cli.download("en_core_web_sm")##newline
eng = spacy.load("en_core_web_sm")
from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer("basic_english")
def engTokenize(text):
    """
    Tokenize an English text and return a list of tokens
    """
    return [token.text for token in eng.tokenizer(text)]

def tokenize(data):
    for x in data:
        yield engTokenize(x)

In [ ]:
# vocab_train = build_vocab_from_iterator(
#     tokenize(trainx),
#     min_freq=2,
#     specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
#     special_first=True
# )
# vocab_train.set_default_index(vocab_train['<unk>'])
# vocab_test = build_vocab_from_iterator(
#     tokenize(testx),
#     min_freq=2,
#     specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
#     special_first=True
# )
# vocab_test.set_default_index(vocab_test['<unk>'])


In [6]:
import pickle
import os

if os.path.exists('vocab_train.pkl'):
    print('Load vocab train')
    with open('vocab_train.pkl', 'rb') as vocab_train_file:
        vocab_train = pickle.load(vocab_train_file)
# else:
#     # Save the trained vocab_train
#     print('Save vocab train')
#     with open('vocab_train.pkl', 'wb') as vocab_train_file:
#         pickle.dump(vocab_train, vocab_train_file)

if os.path.exists('vocab_test.pkl'):
    print('Load vocab test')
    # Load vocab_test from the pickle file
    with open('vocab_test.pkl', 'rb') as vocab_test_file:
        vocab_test = pickle.load(vocab_test_file)
# else:
#     print('Save vocab test')
#     # Save the trained vocab_test
#     with open('vocab_test.pkl', 'wb') as vocab_test_file:
#         pickle.dump(vocab_test, vocab_test_file)


print(vocab_train.get_itos()[:45])

Load vocab train
Load vocab test
['<pad>', '<sos>', '<eos>', '<unk>', 'game', ' ', 'like', 'one', 'games', 'get', 'play', 'good', 'really', 'time', 'great', 'fun', 'would', 'much', 'even', 'first', 'also', 'well', 'graphics', 'story', 'new', 'playing', 'way', 'better', 'still', 'make', 'use', 'played', 'many', 'could', 'lot', 'want', 'back', 'little', 'pretty', 'buy', 'gameplay', 'bad', 'people', 'best', 'got']


In [7]:
vocabsize = len(vocab_train)
print(vocabsize)

77587


In [26]:
# Define the dimensions
input_dim = vocabsize # The dimension of your input data (e.g., vocabulary size)
hidden_dim = 256  # Size of the hidden layer
embedding_dim =  128
output_dim = 1  # Two classes: positive and negative
learning_rate = 0.001
epochs = 100
batch_size = 128

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available. Using GPU.")
else:
    device = torch.device("cpu")
    print("GPU not available. Using CPU.")
print(device)

GPU available. Using GPU.
cuda


In [30]:
import torch.autograd as autograd
# Define the classification model
class SentimentClassifier(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(SentimentClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        # self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = nn.LSTM(input_size= embedding_dim,
                                hidden_size=hidden_dim)        
        
        self.fc1 = nn.Linear(hidden_dim, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32,output_dim)
        # self.sm = nn.LogSoftmax(dim=1)
    #     self.hidden = self.init_hidden()

    # def init_hidden(self):
    #     # the first is the hidden h
    #     # the second is the cell  c
    #     return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim, dtype=torch.float)).to(device),
    #             autograd.Variable(torch.zeros(1, 1, self.hidden_dim, dtype=torch.float)).to(device))
    
    def forward(self, text, length):
        # text dim: [sentence length, batch size]
        
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        # Pack the sequences for efficient processing
        packed_data = pack_padded_sequence(embedded, length.sum(1), batch_first=True, enforce_sorted=False)
        output, (hidden, cell) = self.rnn(packed_data)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]
        # unpacked_output, unpacked_length = pad_packed_sequence(output,batch_first=True)
        # unpacked_output.squeeze_(1)
        # hidden dim: [batch size, hidden dim]
        o = self.fc1(hidden.squeeze(0))
        o = self.dropout(o)
        o = self.fc3(o)
        o = self.dropout(o)
        o = self.fc4(o)
        # o = self.sm(o)
        # o = self.sigmoid(o.squeeze(0))
        return o

In [11]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # sample = {'data': self.data[idx], 'label': self.labels[idx]}
        return self.data.iloc[idx], self.labels[idx]



In [12]:
y_train = trainy.values
y_test = testy.values

print(y_train)

[1 1 0 ... 1 1 1]


In [13]:
def getTransform(vocab):
    """
    Create transforms based on given vocabulary. The returned transform is applied to sequence
    of tokens.
    """
    text_tranform = T.Sequential(
        ## converts the sentences to indices based on given vocabulary
        T.VocabTransform(vocab=vocab),
        ## Add <sos> at beginning of each sentence. 1 because the index for <sos> in vocabulary is
        # 1 as seen in previous section
        T.AddToken(1, begin=True),
        ## Add <eos> at beginning of each sentence. 2 because the index for <eos> in vocabulary is
        # 2 as seen in previous section
        T.AddToken(2, begin=False)
    )
    return text_tranform
def applyTransform(text):
    """
    Apply transforms to sequence of tokens in a sequence pair
    """

    return getTransform(vocab_train)(engTokenize(text))

In [14]:
# x_train = trainx.apply(applyTransform)
# x_test = testx.apply(applyTransform)

In [14]:
if os.path.exists('x_train.pkl'):
    print('Load x_train vectors')
    # Load vocab_test from the pickle file
    with open('x_train.pkl', 'rb') as x_train_file:
        x_train = pickle.load(x_train_file)
# else:
#     print('Save x_train vectors')
#     # Save the trained vocab_test
#     with open('x_train.pkl', 'wb') as x_train_file:
#         pickle.dump(x_train, x_train_file)

if os.path.exists('x_test.pkl'):
    print('Load x_test vectors')
    # Load vocab_test from the pickle file
    with open('x_test.pkl', 'rb') as x_test_file:
        x_test = pickle.load(x_test_file)
# else:
#     print('Save x_test vectors')
#     # Save the trained vocab_test
#     with open('x_test.pkl', 'wb') as x_test_file:
#         pickle.dump(x_test, x_test_file)

Load x_train vectors
Load x_test vectors


In [15]:
def custom_collate_fn(batch):
    data, labels = zip(*batch)

    # Sort sequences by length (from longest to shortest)
    sorted_data, sorted_labels = zip(*sorted(zip(data, labels), key=lambda x: len(x[0]), reverse=True))

    # Pad sequences to the length of the longest sequence
    padded_data = pad_sequence([torch.tensor(seq) for seq in sorted_data], batch_first=True)
    padded_labels = torch.tensor(sorted_labels, dtype=torch.float64)

    # Create a mask for the padded elements
    mask = (padded_data != 0).float()

    return padded_data.to(device), mask, padded_labels.to(device)

In [16]:
torch.manual_seed(12)
trainset = CustomDataset(x_train, y_train)
testset = CustomDataset(x_test, y_test)

# trainset = CustomDataset(vocab_train, y_train_onehot)
# testset = CustomDataset(vocab_test, y_test)

# Create a DataLoader for your dataset
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)


In [44]:
# Instantiate the classifier
model = SentimentClassifier(input_dim, embedding_dim, hidden_dim, output_dim).to(device)
# Define the loss function (cross-entropy) and the optimizer (Adam)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
for x,y,z in test_loader:
    print(x)
    print('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA')
    print(y)
    print('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA')
    print(z.unsqueeze(1))
    print('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA')
    # seq_unpacked, lens_unpacked = pad_packed_sequence(x, batch_first=True)
    # print(seq_unpacked)
    break

tensor([[   1,   86, 9462,  ...,  624, 1187,    2],
        [   1,    5,  688,  ...,    0,    0,    0],
        [   1, 1129, 1089,  ...,    0,    0,    0],
        ...,
        [   1,  420,    2,  ...,    0,    0,    0],
        [   1,  249,    2,  ...,    0,    0,    0],
        [   1,   41,    2,  ...,    0,    0,    0]], device='cuda:0')
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
       

In [42]:
torch.manual_seed(76)
total_acc, total_count = 0,0
for x,y,z in test_loader:
    z = z.unsqueeze(1)
    predictions = model(x,y)
    # print(predictions)
    # print(z)
    loss = criterion(predictions,z)
    loss.backward()
    # print(loss.item())
    
    result = (predictions>0).float()
    result = result[torch.all(result==z, dim=1)]
    print(result)
    total_acc += len(result)
    total_count += len(z)
    print(f"{total_acc/total_count*100}%")
    break

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.]], device='cuda:0')
38.28125%


In [43]:
def trainloop(trainloader):
    epoch_train_loss = 0.0
    model.train()
    with tqdm(trainloader, unit="batch", desc='Training: ') as tbatch:
        for data, mask, labels in tbatch:
            labels = labels.unsqueeze(1)
            optimizer.zero_grad()
            output = model(data, mask)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss
            tbatch.set_postfix(loss="{:.6f}".format(loss))
    return epoch_train_loss/len(trainloader)

def evaluateloop(testloader):
    model.eval()
    total_acc, total_count = 0, 0
    with tqdm(testloader, unit="batch", desc='Evaluating: ') as ttest:
        with torch.no_grad():
            for data, mask, labels in ttest:
                labels = labels.unsqueeze(1)
                predictions = model(data, mask)
                result = (predictions>0).float()
                result = result[torch.all(result==labels, dim=1)]
                total_acc += len(result)
                total_count += len(labels)
                ttest.set_postfix({'accuracy': total_acc / total_count})
    accuracy = total_acc / total_count
    return accuracy

In [20]:
def checkpoint(model, optimizer, loss, epoch, filename):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, filename)
def resume(filename):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return loss

In [45]:
start_time = time.time()
torch.manual_seed(0)
last_lost = 0.0 #the lost of the last epoch
patience = 3 
trigger_times = 0 #counter for early stop
stop = False
start_epoch=0
curr_epoch = 0
delta = 0.0002
counter = 0
try:
    with open('loss_values.pkl', 'rb') as f_loss, open('accuracy_values.pkl', 'rb') as f_accuracy:
        loss_array = pickle.load(f_loss)
        accuracy_array = pickle.load(f_accuracy)
except FileNotFoundError:
    # If files don't exist, initialize empty arrays
    loss_array = [] #used for graphing loss changes throughout training
    accuracy_array = [] 


if start_epoch>0:
    print("Found existing checkpoint, continue training")
    last_lost = resume(f'Model_at_epoch_{start_epoch}.pth')

try:
    for epoch in range(start_epoch, epochs):
        curr_epoch = epoch
        #Train
        train_loss = trainloop(train_loader)

        #Test
        accuracy = evaluateloop(test_loader)
        accuracy_array+=[accuracy]
        loss_array+=[train_loss]

        if counter%5==0:
            checkpoint(model, optimizer, loss_array[-1], curr_epoch, f'Model_at_epoch_{curr_epoch}.pth')
            with open('loss_values.pkl', 'wb') as f_loss, open('accuracy_values.pkl', 'wb') as f_accuracy:
                pickle.dump(loss_array, f_loss)
                pickle.dump(accuracy_array, f_accuracy)

        counter+=1
        # Print statistics at the end of each epoch
        print(f'Epoch [{epoch+1}/{epochs}] | Test Accuracy: {accuracy:.2%} | Current Loss: {train_loss: .6f}\n')
        #Early stopping check
        if abs(train_loss - last_lost)<=delta:
            trigger_times+=1
        elif abs(train_loss - last_lost)>delta:
            trigger_times=0
        if trigger_times>patience:
            stop = True
        last_lost=train_loss
        if stop == True:
            break
except KeyboardInterrupt:
    print("Training interrupted. Saving current model checkpoint.")
    checkpoint(model, optimizer, loss_array[-1], curr_epoch, f'Model_at_epoch_{curr_epoch}.pth')
    with open('loss_values.pkl', 'wb') as f_loss, open('accuracy_values.pkl', 'wb') as f_accuracy:
        pickle.dump(loss_array, f_loss)
        pickle.dump(accuracy_array, f_accuracy)

# Print total training time
print("Total training time: {:.2f} seconds".format(time.time() - start_time))
#Save model
checkpoint(model, optimizer, loss_array[-1], curr_epoch, f'Model_at_epoch_{curr_epoch}.pth')
with open('loss_values.pkl', 'wb') as f_loss, open('accuracy_values.pkl', 'wb') as f_accuracy:
    pickle.dump(loss_array, f_loss)
    pickle.dump(accuracy_array, f_accuracy)


Training:   0%|          | 0/2063 [00:00<?, ?batch/s]

Evaluating: 100%|██████████| 617/617 [00:37<00:00, 16.42batch/s, accuracy=0.83] 


NameError: name 'checkpoint' is not defined

In [ ]:
torch.save({
    'epoch': curr_epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss_array[-1],
}, f"Model_at_epoch_{curr_epoch}.pth")

In [34]:
test_sentence = 'I love this game'
test_sentence2 = 'Trash game'
data = [test_sentence, test_sentence2,'Greate game i like it','I regret buying it']
result = [1,0,1,0]
column_names = ['text', 'positive']
df = pd.DataFrame(list(zip(data, result)), columns=column_names)

text = df['text']
label = df['positive']
evaluate = CustomDataset(text.apply(applyTransform),label.values)
evaloader = DataLoader(evaluate,batch_size=1, collate_fn=custom_collate_fn)

In [37]:
torch.manual_seed(0)
for batch, text in zip(evaloader,data):
    x,y,z= batch
    predictions = model(x,y)
#     print(predictions)
    result = predictions.argmax(1)==z
    if result == True:
        print(f'Prediction result: {text} - Correct')
    else:
        print(f'Prediction result: {text} - Incorrect')

Prediction result: I love this game - Correct
Prediction result: Trash game - Incorrect
Prediction result: Greate game i like it - Correct
Prediction result: I regret buying it - Incorrect
